You need to implement Logistic Regression from scratch in this question

1. You are provided with the dataset of sign language digits. Implement logistic regression from scratch to classify the images provided in the dataset. Load the dataset and perform splitting into training and test sets with 70:30 ratio randomly using test train split.
2. Plot a diagram for the sigmoid function. This is used for binary classi cation. How do you modify it for multilabel dataset classification problems? State and Explain the methods used.
3. Use both one vs all and one vs one method for the above problem statement purpose.
4. Also get results using Log Reg from scikit learn.
5. Report accuracy score, Confusion matrix and any other metrics you feel useful and Compare the results - from all the three.



[BONUS]
6. Display few pictures with their predicted and original labels 
7. Do the results differ? State the reasons why it is so.


dataset link : 
https://iiitaphyd-my.sharepoint.com/:f:/g/personal/apurva_jadhav_students_iiit_ac_in/Eictt5_qmoxNqezgQQiMWeIBph4sxlfA6jWAJNPnV2SF9Q?e=mQmYN0 

                

In [94]:
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from scipy import stats

In [2]:
X = np.load("X.npy") # image
Y = np.load("Y.npy") # label
X = np.reshape(X, (X.shape[0], X.shape[1]*X.shape[2]))


In [33]:
Y_temp = np.argmax(Y,axis = 1)

In [36]:
Y = np.append(Y,Y_temp.reshape(Y_temp.shape[0],1),axis = 1)

In [39]:
X_train,X_test,Y_train,Y_test = model_selection.train_test_split(X,Y,test_size = 0.3,random_state =42)

In [40]:
Y_train_label = Y_train[:,-1]
Y_train = Y_train[:,:-1]
Y_test_label = Y_test[:,-1]
Y_test = Y_test[:,:-1]

In [42]:
def preprocess(X):
    X_copy = X.copy()
    b = np.ones(shape=(len(X_copy),1))
    return np.append(X_copy,b,axis=1)

def get_res(X,W):
    return np.dot(X,W)

def sigmoid(Y_pred):
    return (1/(1+np.exp(-Y_pred)))

def update_weights(X,W,Y,learning_rate):
    Y_pred = get_res(X,W)    
    Y_pred = sigmoid(Y_pred)
    N = len(X)
#     print(X.shape,Y.shape,W.shape,Y_pred.shape)
#     return
    gradient = learning_rate * (np.dot(X.T,Y_pred - Y)/N)
    W -= gradient    
    return W

def train(X,Y,learning_rate,itr):
    W = np.zeros((len(Y[0]),len(X[0])+1))
    X_copy = preprocess(X)
    for cls in range(len(Y[0])):
        for i in range(itr):
            W[cls] = update_weights(X_copy,W[cls],Y[:,cls],learning_rate)
    return W

def predict(X,W):
    Y_pred = np.zeros((len(X),10))
    for cls  in range(10):
        Y_pred[:,cls] = sigmoid(get_res(preprocess(X),W[cls]))
    return (np.argmax(Y_pred,axis = 1))

In [43]:
W = train(X_train,Y_train,0.2,3000)

In [44]:
temp = predict(X_test,W)

In [47]:
accuracy_score(Y_test_label,temp)
# One vs All

0.5896607431340872

In [48]:
lr = LogisticRegression(multi_class = 'ovr')
lr.fit(X_train,Y_train_label)


/home/zcruz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/zcruz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.htm

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
yt = lr.predict(X_test)

In [50]:
accuracy_score(Y_test_label,yt)

0.7237479806138933

In [87]:
def train_ovo(X,Y,learning_rate,itr): 
    W = np.zeros(len(X[0])+1)
    X_copy = preprocess(X)
    for i in range(itr):
        W = update_weights(X_copy,W,Y,learning_rate)
    return W

In [88]:
def ovo(X,Y):
    N = len(Y[0])
    W = []
    for i in range(N):
        for j in range(i+1,N):
            clsA = [k for k in range(len(Y[:,i])) if(Y[:,i][k])]
            clsB = [k for k in range(len(Y[:,j])) if(Y[:,j][k])] 
            yA = np.zeros((len(clsA),1))
            yB = np.ones((len(clsB),1))
            ynew = np.append(yA,yB)
            XA = X[clsA]
            XB = X[clsB]
            Xnew = np.append(XA,XB,axis=0)
            data = np.append(Xnew,ynew.reshape(ynew.shape[0],1),axis = 1)
            np.random.shuffle(data)
            X_train = data[:,:-1]
            Y_train = data[:,-1]
            w = train_ovo(X_train,Y_train,0.2,3000)
            W.append({"neg":i,"pos":j,"w":w})
    return W
    
def ovo_predict(X,W):
    Y_pred = np.zeros(shape = (len(X),1))
    for w in W:
        y = sigmoid(get_res(preprocess(X),w["w"]))
        y = np.array([w["pos"] if i > 0.5 else w["neg"] for i in y])
        Y_pred = np.append(Y_pred,y.reshape(y.shape[0],1),axis = 1)
    
    return stats.mode(Y_pred[:,1:],axis = 1)[0]

        
    
    

In [89]:
w_ovo = ovo(X_train,Y_train)

In [90]:
y_ovo_pred = ovo_predict(X_test,w_ovo)

In [91]:
accuracy_score(Y_test_label,y_ovo_pred)
#One vs One

0.7689822294022617

In [95]:
model = LogisticRegression()
ovo_cls = OneVsOneClassifier(model)
ovo_cls.fit(X_train,Y_train_label)

/home/zcruz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/zcruz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.htm

/home/zcruz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/zcruz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.htm

/home/zcruz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/zcruz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.htm

OneVsOneClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='auto', n_jobs=None,
                                                penalty='l2', random_state=None,
                                                solver='lbfgs', tol=0.0001,
                                                verbose=0, warm_start=False),
                   n_jobs=None)

In [97]:
accuracy_score(ovo_cls.predict(X_test),Y_test_label)

0.802907915993538